## Demo Skript to determine single pictures of bunnies from group photos

In [ ]:
import sys
import os

# Add the project root to sys.path so 'config' can be imported
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
import torch
from torchvision import transforms

from PIL import Image, ImageDraw
import os
import pytorch_lightning as pl

from config.load_configuration import load_configuration
config = load_configuration()

pl.seed_everything(config['seed'])
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"   # disable oneDNN optimizations for reproducibility

Seed set to 42


PC Name: DESKTOP-MIKA
Loaded configuration from ../config/config_mika.yaml


In [2]:
import torchvision
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

f:\Users\Mika\Anaconda\envs\VDKI-Projekt\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
f:\Users\Mika\Anaconda\envs\VDKI-Projekt\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
# Helper function to get bounding boxes for rabbits
def get_rabbit_boxes(image, threshold=0.5):                                                         #TODO: adjust threshold as necessary
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    img_tensor = transform(image)
    with torch.no_grad():
        prediction = model([img_tensor])[0]
    boxes = []
    labels = prediction['labels'].tolist()
    scores = prediction['scores'].tolist()
    # print(labels, scores)                                                                         #TODO: show labels and scores
    accepted_labels = [88, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]  # COCO classes for animals      #TODO: add labels as necessary
    # COCO: 17=cat, 18=dog, 19=horse, 20=sheep, 21=cow, 22=elephant, 23=bear, 24=zebra, 25=giraffe
    for box, label, score in zip(prediction['boxes'], labels, scores):
        relative_y_center = (box[1] + box[3]) / 2 / image.height
        relative_x_center = (box[0] + box[2]) / 2 / image.width

        if label in accepted_labels and score > threshold:
            add_box = True
            for boxbox in boxes:
                relative_x_center_boxbox = (boxbox[0] + boxbox[2]) / 2 / image.width
                relative_y_center_boxbox = (boxbox[1] + boxbox[3]) / 2 / image.height
                if abs(relative_x_center - relative_x_center_boxbox) < 0.1 and abs(relative_y_center - relative_y_center_boxbox) < 0.1:
                    # If the center of the new box is too close to an existing box, skip it
                    add_box = False
                    break
            if add_box: boxes.append(box.int().tolist())
        else: print(f"Rejected box with label {label} and score {score:.2f}")
    return boxes

def save_rabbit_crops(image_path, target_path, border=10):
    image = Image.open(image_path).convert("RGB")
    boxes = get_rabbit_boxes(image)
    # print(f"Found {len(boxes)} rabbit-like objects in {image_path}.")
    base, ext = os.path.splitext(os.path.basename(image_path))
    for idx, box in enumerate(boxes):
        image_copy=image.copy()  # Create a copy of the image to avoid modifying the original
        x1, y1, x2, y2 = box
        x1b = x1 - border
        y1b = y1 - border
        x2b = x2 + border
        y2b = y2 + border
        # Make crop quadratic
        width = x2b - x1b
        height = y2b - y1b
        side = max(width, height)
        center_x = (x1b + x2b) // 2
        center_y = (y1b + y2b) // 2
        half_side = side // 2
        x1q = center_x - half_side
        y1q = center_y - half_side
        x2q = center_x + half_side
        y2q = center_y + half_side
        if (x1q < 0 or y1q < 0 or x2q > image.width or y2q > image.height):
            # If the crop goes out of bounds, pad the image
            pad_l = max(0, -x1q)
            pad_t = max(0, -y1q)
            pad_r = max(0, x1q + side - image.width)
            pad_b = max(0, y1q + side - image.height)
            image_copy = transforms.functional.pad(image, (pad_l, pad_t, pad_r, pad_b), padding_mode="reflect")  # Pad the image if the crop goes out of bounds
            x1q = x1q + pad_l
            y1q = y1q + pad_t
            x2q = x2q + pad_l
            y2q = y2q + pad_t
            # print("picture transformed")
        
        image_copy = image_copy.crop((x1q, y1q, x2q, y2q))
        image_copy.save(os.path.join(target_path, f"{base}_crop_{idx}{ext}"))
        image_copy.close()
        # print(f"idx: {idx}")
        # print(f"box: {box}")
        # print(f"x1: {x1}, y1: {y1}, x2: {x2}, y2: {y2}")
        # print(f"x1b: {x1b}, y1b: {y1b}, x2b: {x2b}, y2b: {y2b}")
        # print(f"width: {width}, height: {height}, side: {side}")
        # print(f"center_x: {center_x}, center_y: {center_y}, half_side: {half_side}")
        # print(f"x1q: {x1q}, y1q: {y1q}, x2q: {x2q}, y2q: {y2q}")
    image.close()
    if len(boxes) == 0:
        print(f"No rabbit-like objects found in {base}. No crops saved.")

# Example usage:
# save_rabbit_crops('IMG_7017.jpeg', border=20)
# folder_name = "NyxHelios"
# folder_name = "ApolloSelene"
folder_name = "Apollo"
for folder_name in  ["Aster", "Helios", "Nyx", "NyxAster", "NyxAsterSelene", "NyxHelios", "NyxSelene", "Selene"]:  # List of folder names to process
    base_path = r"F:\Users\Mika\Documents\Studium_HKA\Semester2\HKA_VDKI\Mika_Data\\" + folder_name  # Base path to the dataset
    pathToBunnies = base_path + r"\Sources"  # Directory containing bunny images
    target_path = base_path + r"\Crops"  # Directory to save cropped images
    for filename in os.listdir(pathToBunnies):
        if filename.lower().endswith('.mov'):
            os.remove(os.path.join(pathToBunnies, filename))
            continue
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
            continue
        file_path = os.path.join(pathToBunnies, filename)
        save_rabbit_crops(file_path, target_path, border=30)